# SAWI

This notebook estimates the Simultaneous Accessibility Walking Index (SAWI) for every origin of a territory. The SAWI is defined as
$$
\text{SAWI}_i = \sum_{k} N_{ik}*w_k
$$
where
- $N_i$ is the normalized pedestrian accessibility of origin $i$ for opportunity $k$.
- $w_k$ is the estimated weight of opportunity $k$.

SAWI estimates are conputed for the three weighting methodologies: Equal, PCA and AHP weights.

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
# Import the names of the territories
l_names = pd.read_csv('../data/input/table/mpios_names.txt',header=None)
l_names = list(l_names[0])

# Types of codes
codes_names_list = ['health','sport','education','financial','cultural','parks']

# Import the population data
DANE_data = pd.read_csv('../data/input/table/DANE_2018_personas_manz.txt',low_memory=False)
pop = DANE_data[['MANZ_CCNCT','poblacion']]

In [ ]:
all_blocks = []
for m in l_names:
    # Import blocks
    blocks = gpd.read_file(f'../data/output/shape/blocks/{m}_blocks.shp')
    blocks = blocks[['MANZ_CCNCT','geometry']]
    blocks['territory'] = m

    # Combine the population and the blocks
    blocks = blocks.set_index('MANZ_CCNCT').join(pop.set_index('MANZ_CCNCT'))
    blocks = blocks.reset_index()
    blocks = blocks.rename(columns={'poblacion':'p_(h)'})
    # Change the NaN values for 0.0
    blocks = blocks.fillna(0.0)

    blocks['pp'] = blocks['p_(h)']/blocks['p_(h)'].sum()
    # Create a list with the accessibility type names: health, sport, etc.
    A_i_names = []
    # Create a list with the normalized accessibility type names
    N_i_names = []
    # Create a loop to agregate each accessibility measure to the blocks
    for c in codes_names_list:
        # Import the accessibility DataFrame
        acc_i_df = pd.read_csv(f'../data/output/table/accessibility_dfs/contour/accessibility_i_contour_15min_{m}_{c}.txt')
        # Filter the accessibility DataFrame
        acc_i_df = acc_i_df[['MANZ_CCNCT','Acc_i']]
        A_i_names.append(f'A_i_{c[:3]}')
        N_i_names.append(f'N_i_{c[:3]}')
        # Rename the accessibility column with the respective type
        acc_i_df = acc_i_df.rename(columns={'Acc_i':f'A_i_{c[:3]}'})
        blocks = blocks.merge(acc_i_df,on='MANZ_CCNCT',how='left')
        blocks = blocks.fillna(0.0)
    # Eliminate blocks with 0 population
    blocks_norm = blocks.copy()
    blocks_norm = blocks_norm.drop(blocks_norm[blocks_norm['p_(h)']==0].index).reset_index(drop=True)
    
    # Normalize by territory
    for i in range(len(A_i_names)):
        blocks_norm[N_i_names[i]] = (blocks_norm[A_i_names[i]]-blocks_norm[A_i_names[i]].min())/(blocks_norm[A_i_names[i]].max()-blocks_norm[A_i_names[i]].min())

    blocks_norm = blocks_norm.fillna(0.0)
    blocks_norm = blocks_norm.set_index('MANZ_CCNCT')

    all_blocks.append(blocks_norm)
    
all_blocks = pd.concat(all_blocks, ignore_index=False)
blocks_norm = all_blocks.copy()

## Equal weighting

In [ ]:
EQU_w = pd.DataFrame({i[-3:]:1/len(N_i_names) for i in N_i_names},index=['w']).T
blocks_norm['SAWI_EQU'] = blocks_norm[N_i_names]@EQU_w.T.rename(columns={i[-3:]:i for i in N_i_names}).T

## PCA weighting

In [ ]:
PCA_w = pd.read_csv('../data/output/table/weights/PCA_weights.txt',index_col=0)
blocks_norm['SAWI_PCA'] = blocks_norm[N_i_names]@PCA_w.T.rename(columns={i[-3:]:i for i in N_i_names}).T

## AHP weighting

In [ ]:
AHP_w = pd.read_csv('../data/output/table/weights/AHP_weights.txt',index_col=0)
blocks_norm['SAWI_AHP'] = blocks_norm[N_i_names]@AHP_w.T.rename(columns={i[-3:]:i for i in N_i_names}).T

## Export

In [ ]:
# For all territories
blocks_norm.to_file('../data/output/shape/accessibility/indexes/SAWI/SAWI_all_territories_norm.shp')
blocks_norm_df = blocks_norm.drop('geometry',axis=1)
blocks_norm_df.to_csv('../data/output/table/accessibility_index/indexes/SAWI/SAWI_all_territories_norm.txt')

# For a single territory
for m in l_names:
    blocks_m = blocks_norm[blocks_norm['territory']==m]
    blocks_m.to_file(f'../data/output/shape/accessibility/indexes/SAWI/SAWI_{m}_norm.shp')
    blocks_m_df = blocks_norm_df[blocks_norm_df['territory']==m]
    blocks_m_df.to_csv(f'../data/output/table/accessibility_index/indexes/SAWI/SAWI_{m}_norm.txt')